In [0]:
import os
import numpy as np
import pandas as pd
import random 
import copy
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_absolute_error as MAE
from sklearn import linear_model as lm
from keras.models import Sequential
from keras.layers import Activation, Dense,Bidirectional,Dropout,LSTM
from itertools import chain

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

In [0]:
root_dir = os.path.abspath(".")

In [0]:
os.chdir(os.path.join(os.path.abspath("."),'gdrive','My Drive'))

In [0]:
 original_data= pd.read_excel('YES_BANK.xlsx',
header=None,
index_col=False,
keep_default_na=True,names=['Date','Open','High','Low','Close','Volume']
)


In [0]:
original_data

In [0]:
original_data=original_data.iloc[::-1]

In [0]:
list_=original_data.values.tolist()
original_data=pd.DataFrame(list_)

In [0]:
original_data

In [0]:
original_data.columns=["Date"	,"Open",	"High",	"Low",	"Close",	"Volume"]
original_data['Date'] = pd.to_datetime(original_data['Date'])
original_data.head()

In [0]:
data_split = int(len(original_data)*0.8)
data1 = original_data
data1

In [0]:
def datasetpreparation(original_data):
  data_split = int(len(original_data)*0.8)
  data1 = original_data.drop(['Date','Volume'],axis = 1)
  data1.head()
  data1_input = []
  for i in range(len(data1) - 4):
    list1 = data1[i:i+5].values.tolist()
    data1_input.append(list1)
  #print(np.array(data4_input).shape)
  #(data4_input)
  data1_output = data1.ix[5:,:]
  #data4_output
  train1_input = np.array(data1_input[0:data_split])
  train1_output = np.array(data1_output[0:data_split])
  print(train1_input.shape)
  print(train1_output.shape)
  test1_input = np.array(data1_input[data_split:])
  test1_output = np.array(data1_output[data_split:])
  print(test1_input.shape)
  print(test1_output.shape)
  return train1_input,train1_output,test1_input,test1_output

In [0]:
def LSTMmodel():
  model1 = Sequential()
  model1.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(5, 4)))
  model1.add(Dropout(0.25))
  model1.add(Dense(4))
  model1.compile(optimizer='adam', loss='mae')
  return model1

In [0]:
def modeltrain(train1_input,train1_output,model1):
  #train_input,train_output,test1_input,test1_output = datasetpreparation(original_data)
  #model = LSTMmodel 
  history = model1.fit(train1_input, train1_output, epochs=1, verbose=1)
  model1.save('project_model.h5')

In [0]:
def evaluateval(model1,train1_input,train1_output):
  #train_input,train_output,test1_input,test1_output = datasetpreparation(original_data)
  predict = model1.predict(train1_input)
  predict1 = pd.DataFrame(predict)
  output = pd.DataFrame(train1_output)
  for i in range(4):
    plt.plot(predict1.ix[:,i])
    plt.plot(output.ix[:,i])
    plt.show()

In [0]:
def test(model1,train1_input,train1_output,test1_input,test1_output,i):
  input = test1_input[i].reshape(1,5,4)
  train1_input = np.concatenate((train1_input,input),axis = 0)
  train1_input = train1_input[-20:]
  output = test1_output[i].reshape(1,4)
  train1_output = np.concatenate((train1_output,output),axis = 0)
  train1_output = train1_output[-20:]
  prediction = model1.predict(input)
  return prediction,train1_input,train1_output

In [0]:
def feedback(model1,train1_input,train1_output,test1_input,i):
  model1.fit(train1_input, train1_output, epochs=10, verbose=1)
  prediction = model1.predict(test1_input[i].reshape(1,5,4))
  return prediction

In [0]:
from keras.models import load_model

#train1_input,train1_output,test1_input,test1_output = datasetpreparation(original_data)
#model1 = LSTMmodel()
#modeltrain(train1_input,train1_output,model1)
pred_list = []
pred_list_feedback = []
   

def runprograme(original_data):
  train1_input,train1_output,test1_input,test1_output = datasetpreparation(original_data)
  model1 = LSTMmodel()
  model1 = load_model('YES_BANK.h5')
  #evaluateval(model1,train1_input,train1_output)
  
  for i in range(len(test1_input)):
   pred,train1_input,train1_output = test(model1,train1_input,train1_output,test1_input,test1_output,i)
   
   pred=(pred.tolist())
   pred = list(chain.from_iterable(pred))
   pred_list.append(pred)

   prediction = feedback(model1,train1_input,train1_output,test1_input,i)
   prediction=(prediction.tolist())
   prediction = list(chain.from_iterable(prediction))
   pred_list_feedback.append(prediction)
   #print(pred)
   #print(prediction)
   #print(test1_output[i])
  #print(pred_list)
  #print(pred_list_feedback)  
  return pred_list, pred_list_feedback 
#pred,train1_input,train1_output = test(model1,train1_input,train1_output,test1_input,test1_output,0)

#pred=(pred.tolist())
#pred = list(chain.from_iterable(pred))

#pred_list.append(pred)



In [0]:
pred_list, pred_list_feedback = runprograme(original_data)


In [0]:
print(pred_list)
pred_list=pd.DataFrame(pred_list)
pred_list.columns=['Open','High','Low','Close']
pred_list.to_csv('Yes_bank.csv')


pred_list_feedback=pd.DataFrame(pred_list_feedback)
pred_list_feedback.columns=['Open','High','Low','Close']
pred_list_feedback.to_csv('Yes_bank_feedback.csv')


In [0]:
#predict = model1.predict(test1_input)
    predict1 =pred_list_feedback
    output = pd.DataFrame(test1_output)
    plt.figure(figsize = (24,8))
    plt.title('open value vs time')
    plt.ylabel('Open Value')
    plt.xlabel('number of days')
    #plt.legend(loc = 'best')
    plt.plot(predict1.ix[:,0],label = 'predicted')
    plt.plot(output.ix[:,0],label = 'actual')
    plt.legend(loc = 'best')
    plt.show()
    plt.figure(figsize = (24,8))
    plt.title('high value vs time')
    plt.ylabel('high Value')
    plt.xlabel('number of days')
    #plt.legend(loc = 'best')
    plt.plot(predict1.ix[:,1],label = 'predicted')
    plt.plot(output.ix[:,1],label = 'actual')
    plt.legend(loc = 'best')
    plt.show()
    plt.figure(figsize = (24,8))
    plt.title('low value vs time')
    plt.ylabel('low Value')
    plt.xlabel('number of days')
    #plt.legend(loc = 'best')
    plt.plot(predict1.ix[:,2],label = 'predicted')
    plt.plot(output.ix[:,2],label = 'actual')
    plt.legend(loc = 'best')
    plt.show()
    plt.figure(figsize = (24,8))
    plt.title('close value vs time')
    plt.ylabel('Close Value')
    plt.xlabel('number of days')
    #plt.legend(loc = 'best')
    plt.plot(predict1.ix[:,3],label = 'predicted')
    plt.plot(output.ix[:,3],label = 'actual')
    plt.legend(loc = 'best')
    plt.show()